In [152]:
import pandas as pd ## Обновить pandas до последней версии
import numpy as np
import math
import os
import glob
import mysql.connector
import datetime
from mysql.connector import errorcode
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
from datetime import date
import string

from os import listdir
from os.path import isfile, join


import warnings
warnings.filterwarnings("ignore")

In [153]:
user = 'project_user'
database = 'project_db'

In [154]:
my_path = 'C:\\Users\\User\\Documents\\Python Scripts\\IT\\Untitled Folder' 

#### Блок Калинина Артёма

In [155]:
scores_for_vars = pd.read_csv(my_path + '/output_3/scores_for_vars.csv')
scores_for_vars = scores_for_vars.drop(['Unnamed: 0'], axis = 1)

In [178]:
scores_for_vars.head(5)

,age,age_of_car,children,education,employed_by,family,gender,house_ownership,housing,income,income_type,marital_status,position,Total_score
0,0.000000,0.0,0.0,0.000000,30.0,0.000000,100.0,100.0,100.0,33.333333,50.000000,0.000000,16.666667,8.333333
1,66.666667,0.0,100.0,66.666667,30.0,33.333333,0.0,100.0,100.0,33.333333,50.000000,0.000000,33.333333,37.777778
2,0.000000,0.0,0.0,66.666667,0.0,0.000000,100.0,100.0,100.0,33.333333,50.000000,0.000000,50.000000,16.111111
3,0.000000,0.0,100.0,66.666667,10.0,66.666667,100.0,0.0,0.0,16.666667,33.333333,16.666667,0.000000,25.000000
4,33.333333,0.0,0.0,0.000000,0.0,33.333333,100.0,100.0,100.0,16.666667,0.000000,16.666667,16.666667,16.666667


In [157]:
woe_iv_table = pd.read_csv(my_path + '/output_3/woe_iv_table.csv')
woe_iv_table = woe_iv_table.drop(['Unnamed: 0'], axis = 1)

## Веса определяются по IV

In [158]:
IV_table = woe_iv_table[['VAR_NAME', 'IV']].drop_duplicates().copy()
IV_table = IV_table.reset_index()
IV_table = IV_table.drop(['index'], axis = 1)
IV_table

,VAR_NAME,IV
0,age,0.412556
1,age_of_car,0.128749
2,children,0.063739
3,education,0.069542
4,employed_by,0.719720
5,family,0.110766
6,gender,0.001368
7,house_ownership,0.007766
8,housing,0.008665
9,income,0.136414


In [159]:
IV_table['predicive_power'] = np.where((IV_table.IV < 0.02), 'useless',
                                       np.where((IV_table.IV >= 0.02) & (IV_table.IV <= 0.1), 'weak',
                                               np.where((IV_table.IV > 0.1) & (IV_table.IV <= 0.3), 'medium',
                                                       np.where((IV_table.IV > 0.3) & (IV_table.IV <= 0.5), 'strong', 'suspicious'))))
                                       

In [160]:
IV_table

,VAR_NAME,IV,predicive_power
0,age,0.412556,strong
1,age_of_car,0.128749,medium
2,children,0.063739,weak
3,education,0.069542,weak
4,employed_by,0.719720,suspicious
5,family,0.110766,medium
6,gender,0.001368,useless
7,house_ownership,0.007766,useless
8,housing,0.008665,useless
9,income,0.136414,medium


##### Предоставим пользовтелю задать вес для переменной с strong predictive power, остальные веса подсчитаются автоматически, при этом medium и weak являются релевантными, useless слабый вес - остальное на suspicious

In [161]:
IV_table.predicive_power.value_counts()

medium        4
weak          3
useless       3
strong        2
suspicious    1
Name: predicive_power, dtype: int64

In [170]:
strong = float(input())##Минимильный вес равен 0.12

0.2


In [171]:
n_medium = 4
n_weak = 3
n_useless = 3
n_strong = 2
n_suspicious = 1

In [172]:
medium = (1 - strong * n_strong)/1.5/n_medium
weak = (1 - strong * n_strong - n_medium * medium)/n_weak
useless = (1 - strong * n_strong - n_medium * medium  - n_weak * weak)/n_useless
suspicious = (1 - strong * n_strong - n_medium *  medium - n_weak * weak - n_useless * useless)/n_suspicious
if medium > strong:
    print("Выберите вес для strong выше")

In [173]:
print(' strong=', strong, '\n', 'medium=',  "{0:.2f}".format(medium), '\n', 'weak=', "{0:.2f}".format(weak), '\n',  
      'useless=', "{0:.2f}".format(useless), '\n',  'suspicious=', "{0:.2f}".format(suspicious))
#Такие веса у каждой группы, то есть у каждой группы medium вес 0.11, поэтому данном принте сумма не будет 1

 strong= 0.2 
 medium= 0.10 
 weak= 0.07 
 useless= 0.00 
 suspicious= 0.00


In [174]:
print(strong * n_strong + weak * n_weak + n_medium * medium + useless * n_useless + n_suspicious * suspicious)

1.0


In [175]:
IV_table

,VAR_NAME,IV,predicive_power
0,age,0.412556,strong
1,age_of_car,0.128749,medium
2,children,0.063739,weak
3,education,0.069542,weak
4,employed_by,0.719720,suspicious
5,family,0.110766,medium
6,gender,0.001368,useless
7,house_ownership,0.007766,useless
8,housing,0.008665,useless
9,income,0.136414,medium


In [176]:
scores_for_vars['Total_score'] = (scores_for_vars.age * strong + scores_for_vars.age_of_car * medium + 
                                  scores_for_vars.children * weak + scores_for_vars.education * weak + 
                                  scores_for_vars.employed_by * suspicious + scores_for_vars.family * medium + 
                                  scores_for_vars.gender * useless + scores_for_vars.house_ownership * useless + 
                                  scores_for_vars.housing * useless + scores_for_vars.income * medium + 
                                  scores_for_vars.income_type * weak + scores_for_vars.marital_status * strong + 
                                  scores_for_vars.position * medium)

In [177]:
scores_for_vars['Total_score'].describe()

count    60.000000
mean     25.833333
std       9.251943
min       7.222222
25%      18.750000
50%      25.000000
75%      31.805556
max      50.555556
Name: Total_score, dtype: float64

In [180]:
scores_for_vars.head()

,age,age_of_car,children,education,employed_by,family,gender,house_ownership,housing,income,income_type,marital_status,position,Total_score
0,0.000000,0.0,0.0,0.000000,30.0,0.000000,100.0,100.0,100.0,33.333333,50.000000,0.000000,16.666667,8.333333
1,66.666667,0.0,100.0,66.666667,30.0,33.333333,0.0,100.0,100.0,33.333333,50.000000,0.000000,33.333333,37.777778
2,0.000000,0.0,0.0,66.666667,0.0,0.000000,100.0,100.0,100.0,33.333333,50.000000,0.000000,50.000000,16.111111
3,0.000000,0.0,100.0,66.666667,10.0,66.666667,100.0,0.0,0.0,16.666667,33.333333,16.666667,0.000000,25.000000
4,33.333333,0.0,0.0,0.000000,0.0,33.333333,100.0,100.0,100.0,16.666667,0.000000,16.666667,16.666667,16.666667
